In [14]:
from ipyleaflet import Map, basemaps, CircleMarker
from ipywidgets import HTML
import pandas as pd



from ipyleaflet import Icon, Marker, MarkerCluster, Popup

# First is latitude and second is longitude; both in degrees


m = Map(interpolation='nearest', basemap=basemaps.Stamen.Terrain,world_copy_jump=True)

markers = ()


popup = Popup(
            name='popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
def mouseover_callback(marker,location, html):
    def callback(*args, **kwargs):
        popup.child = html
        popup.location = location
        m.add_layer(popup)
    return callback

def mouseout_callback(marker, location, html):
    def callback(*args, **kwargs):
        popup.child = html
        popup.location = location
        m.remove_layer(popup)
    return callback

from data.labeled import LABELED_CITIES as cities, RISKS_MAPPING as risks
import numpy as np
for _,city in cities.iterrows():
    location=(city['latitude'],city['longitude'])
    html =  f"""
    <p> <h4><b> {city.city}, {city.country} </b></h4>
    
    """
    
    for risk in risks:
        risk_value = city[risk]
        if not pd.isnull(risk_value):
            html += f"""
      <h4><b>{risks[risk]}</b>:{risk_value}</h4>
   
    """
    html += '</p>'
    marker = CircleMarker(location=location,fill_color = "blue",color='blue',radius=2)
    marker.on_mouseover(mouseover_callback(marker,location, HTML(html)))
    marker.on_mouseout(mouseout_callback(marker,location, HTML(html)))
    markers = markers + (marker,)

m.add_layer(MarkerCluster(markers = markers))

In [15]:
m.layout.width = '80%'
m.layout.height = '1000px'
m.zoom=2.2
m.center = (0, 0)
display(m)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…